In [ ]:
import lasio_utilities as lasutil
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

In [15]:
# las files
lasfiles = lasutil.loader_lasio_multiprocess("..\\data\\550", outdf=False)
lasfiles.keys()

In [ ]:
gg_df = lasfiles['gg_THK_curves'].df().join(lasfiles['gg_THK_radius'].df())

In [ ]:
gg_df.to_pickle('..\data\gg_df_gz.pkl', compression='gzip')

In [ ]:
gg_df = pd.read_pickle('..\data\gg_df_gz.pkl', compression='gzip')

In [ ]:
gg_df.shape

In [ ]:
#Resampling
#df = df.reindex(df.index.union(resampling))

#Interpolation technique to use. One of:

#'linear': Ignore the index and treat the values as equally spaced. This is the only method supported on MultiIndexes.
#'time': Works on daily and higher resolution data to interpolate given length of interval.
#'index', 'values': use the actual numerical values of the index.
#'pad': Fill in NaNs using existing values.
#'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'spline', 'barycentric', 'polynomial': Passed to scipy.interpolate.interp1d. These methods use the numerical values of the index. Both 'polynomial' and 'spline' require that you also specify an order (int), e.g. df.interpolate(method='polynomial', order=5).
#'krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima': Wrappers around the SciPy interpolation methods of similar names. See Notes.
#'from_derivatives': Refers to scipy.interpolate.BPoly.from_derivatives which replaces 'piecewise_polynomial' interpolation method in scipy 0.18.
# new_index = np.linspace(gg_df.index.min(),gg_df.index.max(),gg_df.shape[0]//10)
new_index = np.arange(gg_df.index.min(),gg_df.index.max(),10)
gg_df_resampled = gg_df.reindex(gg_df.index.union(new_index)).interpolate('values').loc[new_index]

In [ ]:
gg_df_resampled['Classification']

In [ ]:
def build_log_graph(df):
    logplot = make_subplots(rows=1, cols=4, shared_yaxes=True, horizontal_spacing=0.01, specs=[[{'secondary_y': False}, {'secondary_y': False}, {'secondary_y': False}, {'secondary_y': False}]], print_grid=True, shared_xaxes=False)
    
    logplot.update_xaxes(showline=True, linewidth=2, mirror=True, ticks='inside', tickangle=0)
    logplot.update_yaxes(showline=True, linewidth=2,
                         mirror=True, ticks='outside')
    # for i in range(len(logs)):

    #     if i == 1:
    #         logplot.add_trace(go.Scatter(x=dataframes[plot_chosen][logs[i]], y=dataframes[plot_chosen]['DEPTH_MD'],
    #                                      name=logs[i], line_color=colors[i]), row=1, col=log_cols[i])
    #         logplot.update_xaxes(type='log', row=1, col=log_cols[i], title_text=logs[i],
    #                              tickfont_size=12, linecolor='#585858')

    #     else:
    #         logplot.add_trace(go.Scatter(x=dataframes[plot_chosen][logs[i]], y=dataframes[plot_chosen]['DEPTH_MD'],
    #                                      name=logs[i], line_color=colors[i]), row=1, col=log_cols[i])
    #         logplot.update_xaxes(col=log_cols[i], title_text=logs[i], linecolor='#585858')

    logplot.add_trace(go.Scatter(x=df.ECCENTRICITY, y=df.index, name=df.ECCENTRICITY.name, line_color='red', x0=0, dx=0.1), row=1, col=1)
    logplot.update_xaxes(row=1, col=1, title_text='ECCENTRICITY (in)', range=[0, 0.5], nticks=6)

    # some adjustmentns for xaxis3
    logplot.update_layout(xaxis12=dict(
            title="OVALITY (%)",
            # titlefont=dict(
            #     color="blue"
            # ),
            # tickfont=dict(
            #     color="blue"
            # ),
            anchor="free",
            overlaying="x",
            side="bottom",
            position=0,
            range=[0, 5],
            #tickvals=[0, 1, 2, 3, 4, 5],
            nticks=6,
            mirror=True,
            showline=True,
            ticks="inside",
        ))

    logplot.add_trace(go.Scatter(x=df.OVALITY, y=df.index, name=df.OVALITY.name, xaxis='x12', line_color='blue'))
    #logplot.data[1].update(xaxis='x11')
    # logplot.layout.xaxis.update({'anchor': 'y', 'overlaying': 'x', 'side': 'top'})

    logplot.add_trace(go.Scatter(x=df.MINID, y=df.index, name=df.MINID.name, line_color='navy'), row=1, col=2)
    logplot.add_trace(go.Scatter(x=df.MEANID, y=df.index, name=df.MEANID.name, line_color='grey'), row=1, col=2)
    logplot.add_trace(go.Scatter(x=df.MAXID, y=df.index, name=df.MAXID.name, line_color='crimson'), row=1, col=2)
    logplot.update_xaxes(row=1, col=2, title_text='ID')
    
    logplot.add_trace(go.Scatter(x=df.MINPEN, y=df.index, name=df.MINPEN.name, line_color='navy'), row=1, col=3)
    logplot.add_trace(go.Scatter(x=df.MEANPEN, y=df.index, name=df.MEANPEN.name, line_color='darkgray'), row=1, col=3)
    logplot.add_trace(go.Scatter(x=df.MAXPEN, y=df.index, name=df.MAXPEN.name, line_color='crimson'), row=1, col=3)
    logplot.update_xaxes(row=1, col=3, title_text='Penetration', range=[-0.1, 0.1])

    logplot.add_trace(go.Heatmap(z=df[[fing for fing in df.columns if 'FING' in fing]], y=df.index, colorscale=[[0.0, "rgb(0,0,0)"],[0.75, "rgb(255,255,255)"],[1.0, "rgb(255, 104, 32)"]], name='Image', x=np.arange(0,361, 1.879901)), row=1, col=4)
    logplot.update_xaxes(row=1, col=4, title_text='Radial Measurement', range=[0, 360])
    logplot.update_yaxes(row=1, col=1, autorange='reversed')
    logplot.update_layout(height=750, width=1300, showlegend=True, yaxis_title='Depth (m)', template='plotly_dark')

    logplot.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.01,
    xanchor="left",
    x=0
    ))

    # make room to display double x-axes
    logplot.update_layout(yaxis1=dict(domain=[0.12, 1]),
                    yaxis2=dict(domain=[0.12, 1]),
                    yaxis3=dict(domain=[0.12, 1]),
                    yaxis4=dict(domain=[0.12, 1]),
                    )

    # # not critical, but just to put a little air in there
    # logplot.update_layout(xaxis1=dict(domain=[0.0, 0.4]),
    #                 xaxis2=dict(domain=[0.6, 1]),
    #                 )

    return logplot

build_log_graph(gg_df_resampled)

In [ ]:
gg_df['CLASSIFICATION'].where((gg_df['CLASSIFICATION'] < 1.0) & (gg_df['CLASSIFICATION'] > -999.25), 0.0, inplace=True)

In [ ]:
# gg_df['MINIDPHASE_FLAG'] = gg_df['MINIDPHASE'].loc[gg_df['OVALITY'] > 2]
# gg_df['MAXIDPHASE _FLAG'] = gg_df['MAXIDPHASE'].loc[gg_df['OVALITY'] > 2]

# gg_df['MINIDPHASE'] = pd.Series(gg_df['MINIDPHASE'].rolling(window=32, min_periods=1, center=True).mean())
# gg_df['MAXIDPHASE'] = pd.Series(gg_df['MAXIDPHASE'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['OVALITY'] = pd.Series(gg_df['OVALITY'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['ECCENTRICITY'] = pd.Series(gg_df['ECCENTRICITY'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['MINID'] = pd.Series(gg_df['MINID'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['MAXID'] = pd.Series(gg_df['MAXID'].rolling(window=32, min_periods=1, center=True).mean())
gg_df['MEANID'] = pd.Series(gg_df['MEANID'].rolling(window=32, min_periods=1, center=True).mean())

# gg_df['MINIDPHASE'] = gg_df['MINIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MAXIDPHASE'] = gg_df['MAXIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['OVALITY'] = gg_df['OVALITY'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['ECCENTRICITY'] = gg_df['ECCENTRICITY'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MINID'] = gg_df['MINID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MAXID'] = gg_df['MAXID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MEANID'] = gg_df['MEANID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MINIDPHASE'] = gg_df['MINIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]
# gg_df['MAXIDPHASE'] = gg_df['MAXIDPHASE'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]


# gg_df['MINIDPHASE'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXIDPHASE'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MINID'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MEANID'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXID'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXPENJOINT'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['CLASSIFICATION'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MINPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MINPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MEANPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['MAXPEN'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['ECCENTRICITY'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)
# gg_df['OVALITY'].where((gg_df.index > 17000) & (gg_df.index < 17200), pd.NA , inplace=True)

In [ ]:
gg_df['MEANID'] = gg_df['MEANID'].loc[(gg_df.index < 16940) | (gg_df.index > 17300)]

In [ ]:
gg_df['CLASSIFICATION'].plot(template='plotly_dark')

In [ ]:
gg_df['MEANID'].plot(template='plotly_dark')

In [ ]:
window_size=64
threshold=100
min_periods=1
df = gg_df.copy()

for colname in df.columns:
    smothy = pd.Series(df[colname].rolling(window=window_size, min_periods=min_periods, center=True).mean())
    difference = pd.DataFrame.abs(df[colname] - smothy)
    outliers = difference > threshold
    #df[colname].where(outliers, smothy, inplace=True)
    df[colname] = pd.Series([original if outliers else despiked for original, despiked, outliers in zip(df[colname], smothy, outliers)])


In [ ]:
def smothy_data(df_org, window_size=64, threshold=100, min_periods=1):
    """
    Smothy data using rolling mean and thresholding
    """
    df = df_org.copy()
    for colname in df.columns:
        smothy = pd.Series(df[colname].rolling(window=window_size, min_periods=min_periods, center=True).mean())
        difference = pd.DataFrame.abs(df[colname] - smothy)
        outliers = difference > threshold
        #df[colname].where(outliers, smothy, inplace=True)
        df[colname] = pd.Series([original if outliers else despiked for original, despiked, outliers in zip(df[colname], smothy, outliers)])
    return df

    #return pd.Series([original if outliers else despiked for original, despiked, outliers in zip(df[colname], smothy, outliers)])

gg_df = smothy_data(gg_df, window_size=32, threshold=0.01, min_periods=1)

In [ ]:
lasfiles['gg_2_curves'].set_data(gg_df)
lasfiles['gg_2_curves'].write('gg_2_curves_FIXED.las', version=2.0)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[1, 2, 3],
    y=[4, 5, 6],
    name="xaxis1 data"
))


fig.add_trace(go.Scatter(
    x=[2, 3, 4],
    y=[40, 50, 60],
    name="xaxis2 data",
    xaxis="x2"
))

fig.add_trace(go.Scatter(
    x=[4, 5, 6],
    y=[40000, 50000, 60000],
    name="xaxis3 data",
    xaxis="x3"
))

fig.add_trace(go.Scatter(
    x=[5, 6, 7],
    y=[400000, 500000, 600000],
    name="xaxis4 data",
    xaxis="x4"
))


# Create axis objects
fig.update_layout(
    # yaxis=dict(
    #     domain=[0.3, 0.7]
    # ),
    xaxis=dict(
        title="xaxis title",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    xaxis2=dict(
        title="xaxis2 title",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="x",
        side="bottom",
        position=0.15
    ),
    xaxis3=dict(
        title="xaxis3 title",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="y",
        overlaying="x",
        side="top"
    ),
    xaxis4=dict(
        title="xaxis4 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="x",
        side="bottom",
        position=0
    )
)

# Update layout properties
fig.update_layout(
    title_text="multiple x-axes example",
    width=800,
)

fig.show()

In [ ]:
# initial subplot with two traces
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=[1, 2, 3], y=[4, 5, 6]),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=[20, 30, 40], y=[50, 60, 70]),
    row=1, col=2
)

fig.update_layout(height=600, width=800,
                  title_text="Subplots with shared x-axes")

# extra data where xaxis3 is shared with subplot 1
fig.add_trace(go.Scatter(
    x=[11, 12, 13],
    y=[6, 5, 4],
    name="xaxis3 data",
    xaxis="x3"
))

# some adjustmentns for xaxis3
fig.update_layout(xaxis3=dict(
        title="xaxis3 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="x1",
        side="right",
        position=0.0
    ))

# extra data where xaxis4 is shared with subplot 2
fig.add_trace(go.Scatter(
    x=[50, 60, 70],
    y=[60, 60, 60],
    name="xaxis4 data",
    xaxis="x4",
    yaxis = 'y2'
))

# some adjustments for xaxis4
fig.update_layout(xaxis4=dict(
        title="xaxis4 title",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="x2",
        side="right",
        position=0.0
    ))

# make room to display double x-axes
fig.update_layout(yaxis1=dict(domain=[0.1, 1]),
                  yaxis2=dict(domain=[0.1, 1]),
                 )

# not critical, but just to put a little air in there
fig.update_layout(xaxis1=dict(domain=[0.0, 0.4]),
                  xaxis2=dict(domain=[0.6, 1]),
                 )

fig.show()